# Notebook 4: End‑to‑End App Development 🛠️  
![Catering Diagram](https://source.unsplash.com/featured/?catering,diagram)

## Scenario Overview  
*Customer selects menu items (tags: price, cuisine, dietary).*  
We must: 1️⃣ scale recipes → ingredients list, 2️⃣ schedule prep, 3️⃣ produce place‑cards.

```mermaid
flowchart TD
  A[Customer Order] --> B[Reasoning Model<br>Plan];
  B --> C[GPT‑4o Mini<br>Create Prompts];
  C --> D[Worker GPTs<br>Shopping List & Schedule];
  D --> E[Output Docs]
```


## Recipe 7 — Metaprompting the Plan
### 1️⃣ Goal
Use **o4‑mini** to break high‑level plan into JSON subtasks and craft GPT prompts.

In [ ]:
import json, os
sample_order = {
    "delivery_time":"2025-05-15T18:00:00",
    "guests":40,
    "items":["Veg Biryani","Paneer Tikka","Gulab Jamun"]
}
meta_prompt = f"""You are a senior catering planner.
Given the JSON order below, output JSON with keys:
tasks[], gpt_prompt_for_each_task.
Order: {json.dumps(sample_order)}"""
plan = pretty_chat_openai("o4-mini", meta_prompt, temperature=0.2).choices[0].message.content
print(plan)


**Take‑away** – Reasoning model produces *meta‑prompts* that downstream GPT models execute.

**Exercise:** add a cost‑estimation task.

## Recipe 8 — Model Chaining in Action

In [ ]:
import ast, openai, time
plan_dict = ast.literal_eval(plan)  # simplistic parse
results={}
for task, prompt in zip(plan_dict["tasks"], plan_dict["gpt_prompt_for_each_task"]):
    print("▶", task)
    res = pretty_chat_openai("gpt-4o-mini", prompt)
    results[task] = res.choices[0].message.content
# Store or post‑process results...


**Take‑away** – Decouple planning (reasoning) from execution (GPT).

**Exercise:** Insert verification step between chain calls.